能不能讓框架幫我們找出最合適的策略參數組合呢？讓我們來重購策略，讓 Python 幫我們找出最合適的參數組合

### 建立框架運行環境

In [ ]:
!pip install backtrader backtrader_plotting

In [ ]:
import os
import datetime
import backtrader as bt

In [ ]:
%matplotlib inline

### 引入策略

In [ ]:
from shutil import copyfile

copyfile(src = "../input/strategies/OptGoldenCross.py", dst = "../working/OptGoldenCross.py")

In [ ]:
from OptGoldenCross import OptGoldenCross

### 設定初始環境

In [ ]:
# 檢視輸出、開啟多核運算
cerebro = bt.Cerebro(optreturn=False, maxcpus=None)
cerebro.broker.setcash(1e5)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
cerebro.broker.setcommission(commission=0.000945)

In [ ]:
csv_file = os.path.abspath('../input/quantitative-trading/SPY.csv')
feed = bt.feeds.YahooFinanceCSVData(
    dataname=csv_file,
    fromdate=datetime.datetime(2000, 1, 1),
    todate=datetime.datetime(2019, 12, 31),
    reverse=False
)
cerebro.adddata(feed)

### 設定策略優化器

In [ ]:
strats = cerebro.optstrategy(
    OptGoldenCross,
    slow=range(50, 201, 5),
    fast=range(5, 51, 5)
)

In [ ]:
opt_runs = cerebro.run()

## 找出最佳參數組合

In [ ]:
final_results_list = []
for run in opt_runs:
    for strategy in run:
        profit = strategy.profit
        final_results_list.append({
            'Slow': strategy.p.slow,
            'Fast': strategy.p.fast,
            'Profit': profit
        })

final_results_list = sorted(final_results_list, key=lambda x: x['Profit'], reverse=True)

for index, result in enumerate(final_results_list[:5]):
    print(f"# No. {index + 1} > 投資參數 Slow {result.get('Slow')} Fast {result.get('Fast')} 獲利 {round(result.get('Profit'), 2)} 💲")